# **TensorFlow 2.0**

In [1]:
import os
from glob import glob
from datetime import datetime

import numpy as np

import tensorflow as tf
from tensorflow.keras import layers

from tensorflow.keras import datasets 

import matplotlib.pyplot as plt

%matplotlib inline

# **Hyperparameter Tunning**

In [2]:
num_epochs = 50
batch_size = 32

learning_rate = 0.001

dropout_rate = 0.5

input_shape = (32, 32, 3)
num_classes = 10

# **Build Model**

In [3]:
inputs = layers.Input(input_shape)

net = layers.Conv2D(32, (3, 3), padding='SAME')(inputs)
net = layers.Activation('relu')(net)
net = layers.Conv2D(32, (3, 3), padding='SAME')(net)
net = layers.Activation('relu')(net)
net = layers.MaxPooling2D(pool_size=(2, 2))(net)
net = layers.Dropout(dropout_rate)(net)

net = layers.Conv2D(64, (3, 3), padding='SAME')(net)
net = layers.Activation('relu')(net)
net = layers.Conv2D(64, (3, 3), padding='SAME')(net)
net = layers.Activation('relu')(net)
net = layers.MaxPooling2D(pool_size=(2, 2))(net)
net = layers.Dropout(dropout_rate)(net)

net = layers.Flatten()(net)
net = layers.Dense(512)(net)
net = layers.Activation('relu')(net)
net = layers.Dropout(dropout_rate)(net)
net = layers.Dense(num_classes)(net)
net = layers.Activation('softmax')(net)

model = tf.keras.Model(inputs=inputs, outputs=net, name='Basic_CNN')

In [4]:
# Model is the full model w/o custom layers
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate),  # Optimization
              loss='sparse_categorical_crossentropy',  # Loss Function 
              metrics=['accuracy'])  # Metrics / Accuracy

# **Data Preprocess**

In [5]:
train_paths = glob('D:/dataset/cifar/train/*.png')[:1000]
test_paths = glob('D:/dataset/cifar/test/*.png')[:1000]

In [6]:
def get_class_name(path):
    return path.split('_')[-1].replace('.png', '')

In [7]:
train_labels = [get_class_name(path) for path in train_paths]
class_names = np.unique(train_labels)

In [8]:
def get_label(path):
    fname = tf.strings.split(path, '_')[-1]
    lbl_name = tf.strings.regex_replace(fname, '.png', '')
    onehot = tf.cast(lbl_name == class_names, tf.uint8)
    return tf.argmax(onehot)  # 이번에는 onehot이 아닌 label 번호로

In [9]:
def load_image_label(path):
    gfile = tf.io.read_file(path)
    image = tf.io.decode_image(gfile)
    image = tf.cast(image, tf.float32) / 255.  # rescale
    
    label = get_label(path)
    return image, label

In [10]:
def image_preprocess(image, label):
    image = tf.image.random_flip_up_down(image)
    image = tf.image.random_flip_left_right(image)
    return image, label

In [11]:
AUTOTUNE = tf.data.experimental.AUTOTUNE

In [12]:
train_dataset = tf.data.Dataset.from_tensor_slices(train_paths)
train_dataset = train_dataset.map(load_image_label, num_parallel_calls=AUTOTUNE)
train_dataset = train_dataset.map(image_preprocess, num_parallel_calls=AUTOTUNE)
train_dataset = train_dataset.batch(batch_size)
train_dataset = train_dataset.shuffle(buffer_size=len(train_paths))
train_dataset = train_dataset.repeat()

In [13]:
test_dataset = tf.data.Dataset.from_tensor_slices(test_paths)
test_dataset = test_dataset.map(load_image_label, num_parallel_calls=AUTOTUNE)
test_dataset = test_dataset.batch(batch_size)
test_dataset = test_dataset.repeat()

# **Checkpoint**

In [14]:
steps_per_epoch = len(train_paths) // batch_size
validation_steps = len(test_paths) // batch_size

#history 하기전에 모델에 변수로 저장해야한다.
history = model.fit_generator(
    train_dataset,
    steps_per_epoch=steps_per_epoch,
    validation_data=test_dataset,
    validation_steps=validation_steps,
    epochs=num_epochs
)

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/50
31/31 [==============================] - 4s 116ms/step - loss: 2.3136 - accuracy: 0.1219 - val_loss: 2.2988 - val_accuracy: 0.0988
Epoch 2/50
31/31 [==============================] - 3s 113ms/step - loss: 2.2592 - accuracy: 0.1498 - val_loss: 2.1812 - val_accuracy: 0.1815
Epoch 3/50
31/31 [==============================] - 4s 120ms/step - loss: 2.1783 - accuracy: 0.1653 - val_loss: 2.1997 - val_accuracy: 0.1522
Epoch 4/50
31/31 [==============================] - 3s 108ms/step - loss: 2.1894 - accuracy: 0.1860 - val_loss: 2.1656 - val_accuracy: 0.1996
Epoch 5/50
31/31 [==============================] - 3s 107ms/step - loss: 2.0895 - accuracy: 0.2087 - val_loss: 2.0383 - val_accuracy: 0.2379
Epoch 6/50
31/31 [==============================] - 3s 107ms/step - loss: 2.0490 - accuracy: 0.2376 - val_loss: 2.0541 - val_accuracy: 0.2339
Epoch 7/50
31/31 [==============================] - 4s 117ms/step - loss

# **Saving Model**

In [15]:
save_path = 'my_model.h5'

In [16]:
model.save(save_path, include_optimizer=True)
#include_optimizer = True를 주면 오티마이져까지 모델을 저장 시킨다. 

In [17]:
model = tf.keras.models.load_model('my_model.h5') # 모델 불러오기

# **Save Model -  2**
weight 값만 저징하기

In [18]:
model.save_weights('model_weights.h5')

In [20]:
# json 형태로 불러오기
with open('model_architechure.json', 'w') as f:
    f.write(model.to_json())

In [21]:
model.load_weights('model_weights.h5')


model.load_weight('checkpoins')이런 식으로 
checkpoint가 담긴 폴더를 지정해주시면 checkpoint도 불러올 수 있습니다.

# H5 모델 들여다 보기

In [22]:
import h5py

In [23]:
model_file = h5py.File('my_model.h5', 'r+')

In [24]:
model_file.keys()

<KeysViewHDF5 ['model_weights', 'optimizer_weights']>

In [25]:
model_file['model_weights'].keys()

<KeysViewHDF5 ['activation', 'activation_1', 'activation_2', 'activation_3', 'activation_4', 'activation_5', 'conv2d', 'conv2d_1', 'conv2d_2', 'conv2d_3', 'dense', 'dense_1', 'dropout', 'dropout_1', 'dropout_2', 'flatten', 'input_1', 'max_pooling2d', 'max_pooling2d_1']>

In [28]:
model_file['model_weights']['conv2d']['conv2d'].keys()

<KeysViewHDF5 ['bias:0', 'kernel:0']>

In [ ]:
model_file['model_weights']['conv2d']['conv2d']['k']